In [1]:
!pip install transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [2]:
!pip install datasets==3.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [3]:
import torch
import numpy as np
import pandas as pd

In [1]:
### import data

In [4]:
train_df = pd.read_csv("/content/train.tsv", sep='\t', header=None)
train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10235,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report"""
10236,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview
10237,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate
10238,2253.json,false,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...


In [9]:
train_df[1].unique()

array(['false', 'half-true', 'mostly-true', 'true', 'barely-true',
       'pants-fire'], dtype=object)

In [5]:
train_df.shape

(10240, 14)

In [6]:
test_df = pd.read_csv("/content/test.tsv", sep='\t', header=None)
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,11972.json,true,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview
1,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
2,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
3,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
4,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28,23,38,34,7,a news conference
1263,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2,0,0,1,0,"on ABC's ""This Week"""
1264,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1,0,0,0,0,a commentary in The Providence Journal
1265,3186.json,barely-true,Says an EPA permit languished under Strickland...,"environment,government-efficiency",john-kasich,"Governor of Ohio as of Jan. 10, 2011",Ohio,republican,9,8,10,18,3,a news conference


In [10]:
valid_df = pd.read_csv("/content/valid.tsv", sep='\t', header=None)
valid_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,12134.json,barely-true,We have less Americans working now than in the...,"economy,jobs",vicky-hartzler,U.S. Representative,Missouri,republican,1,0,1,0,0,an interview with ABC17 News
1,238.json,pants-fire,"When Obama was sworn into office, he DID NOT u...","obama-birth-certificate,religion",chain-email,NaN,NaN,none,11,43,8,5,105,NaN
2,7891.json,false,Says Having organizations parading as being so...,"campaign-finance,congress,taxes",earl-blumenauer,U.S. representative,Oregon,democrat,0,1,1,1,0,a U.S. Ways and Means hearing
3,8169.json,half-true,Says nearly half of Oregons children are poor.,poverty,jim-francesconi,Member of the State Board of Higher Education,Oregon,none,0,1,1,1,0,an opinion article
4,929.json,half-true,On attacks by Republicans that various program...,"economy,stimulus",barack-obama,President,Illinois,democrat,70,71,160,163,9,interview with CBS News
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,3419.json,half-true,"For the first time in more than a decade, impo...","energy,oil-spill,trade",barack-obama,President,Illinois,democrat,70,71,160,163,9,a press conference
1280,12548.json,mostly-true,Says Donald Trump has bankrupted his companies...,candidates-biography,hillary-clinton,Presidential candidate,New York,democrat,40,29,69,76,7,a speech on the economy
1281,401.json,true,"John McCain and George Bush have ""absolutely n...",health-care,campaign-defend-america,NaN,"Washington, D.C.",none,0,1,0,2,0,a television ad
1282,1055.json,false,A new poll shows 62 percent support the presid...,health-care,americans-united-change,NaN,NaN,none,1,4,4,1,0,an Internet ad.


In [11]:
valid_df.shape

(1284, 14)

In [12]:
test_df.shape

(1267, 14)

In [13]:
# import dataset
# ========================================
import pandas as pd
from datasets import Dataset

def load_and_prepare_data(train_file, val_file, test_file):
    """
    Load LIAR dataset from local TSV files and prepare it for binary classification.

    Binary mapping:
        0 = real (mostly-true, true)
        1 = fake (pants-fire, false, barely-true, half-true)
    """
    # Load TSVs without header
    df_train = pd.read_csv(train_file, sep='\t', header=None)
    df_val   = pd.read_csv(val_file, sep='\t', header=None)
    df_test  = pd.read_csv(test_file, sep='\t', header=None)

    # Assign proper column names (LIAR structure)
    columns = [
        "id",          # 0
        "label_text",  # 1
        "statement",   # 2
        "subject",     # 3
        "speaker",     # 4
        "job_title",   # 5
        "state_info",  # 6
        "party_affiliation", # 7
        "barely_true_count",  # 8
        "false_count",        # 9
        "half_true_count",    # 10
        "mostly_true_count",  # 11
        "pants_fire_count",   # 12
        "source"              # 13
    ]

    df_train.columns = columns
    df_val.columns   = columns
    df_test.columns  = columns

    # Binary label mapping
    def map_labels(label_text):
        fake_labels = ["pants-fire", "false", "barely-true", "half-true"]
        real_labels = ["mostly-true", "true"]
        return 1 if label_text in fake_labels else 0

    for df in [df_train, df_val, df_test]:
        df['label'] = df['label_text'].apply(map_labels)
        df.dropna(subset=['label', 'statement'], inplace=True)

    # Keep only statement + label for dataset
    hf_dataset = {
        'train': Dataset.from_pandas(df_train[['statement', 'label']]),
        'validation': Dataset.from_pandas(df_val[['statement', 'label']]),
        'test': Dataset.from_pandas(df_test[['statement', 'label']])
    }

    return hf_dataset

In [14]:
dataset = load_and_prepare_data(
    train_file="/content/train.tsv",
    val_file="/content/valid.tsv",
    test_file="/content/test.tsv"
)

print(dataset['train'][0])

{'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'label': 1}


In [15]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

In [16]:
# 3. Tokenization
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(batch["statement"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = {split: dataset[split].map(tokenize_fn, batched=True) for split in ["train","validation","test"]}

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/10240 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1267 [00:00<?, ? examples/s]

In [17]:
from peft import LoraConfig
from peft import get_peft_model
import evaluate

In [18]:
# Load base model and apply LoRA
num_labels = 2

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(base_model, peft_config)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Metrics
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    prec = precision.compute(predictions=preds, references=labels, average="weighted")
    rec = recall.compute(predictions=preds, references=labels, average="weighted")
    f1_score = f1.compute(predictions=preds, references=labels, average="weighted")
    return {
        "accuracy": acc["accuracy"],
        "precision": prec["precision"],
        "recall": rec["recall"],
        "f1": f1_score["f1"]
    }

In [20]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results-liar",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

In [21]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2383009693.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
# Train
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.635300,0.620504,0.655763,0.666308,0.655763,0.660149
2,0.613200,0.576563,0.703271,0.681712,0.703271,0.672062
3,0.595700,0.577995,0.697040,0.689967,0.697040,0.692778
4,0.578100,0.577837,0.697819,0.686420,0.697819,0.689966
5,0.559900,0.581026,0.695483,0.684663,0.695483,0.688232


TrainOutput(global_step=3200, training_loss=0.5964232063293458, metrics={'train_runtime': 1643.8801, 'train_samples_per_second': 31.146, 'train_steps_per_second': 1.947, 'total_flos': 6828595662028800.0, 'train_loss': 0.5964232063293458, 'epoch': 5.0})

In [23]:
# Save model
save_path = "./fine_tuned_liar_detector"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./fine_tuned_liar_detector/tokenizer_config.json',
 './fine_tuned_liar_detector/special_tokens_map.json',
 './fine_tuned_liar_detector/vocab.json',
 './fine_tuned_liar_detector/merges.txt',
 './fine_tuned_liar_detector/added_tokens.json',
 './fine_tuned_liar_detector/tokenizer.json')

In [24]:
from google.colab import files
import shutil

In [25]:
# Zip the folder
shutil.make_archive("fine_tuned_liar_detector", 'zip', "./fine_tuned_liar_detector")

'/content/fine_tuned_liar_detector.zip'

In [26]:
# Download the zip file
files.download("fine_tuned_liar_detector.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
# Load Models for Inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
from transformers import RobertaTokenizer, AutoModelForSequenceClassification

In [29]:
roberta_tokenizer = RobertaTokenizer.from_pretrained(save_path)
roberta_model = AutoModelForSequenceClassification.from_pretrained(save_path).to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from transformers import AutoModelForSeq2SeqLM

In [31]:
flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [32]:
# Input Parser (Text | URL | PDF)
def extract_text(input_type, value):
    try:
        if input_type == 'url':
            article = newspaper.Article(value)
            article.download()
            article.parse()
            return article.text
        elif input_type == 'pdf':
            with fitz.open(value) as doc:
                return "\n".join([page.get_text() for page in doc])
        elif input_type == 'text':
            return value
        else:
            return "Invalid input type"
    except Exception as e:
        return f"Error processing input: {str(e)}"

In [33]:
!pip install scikit-learn newspaper3k PyMuPDF duckduckgo-search langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 122.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [34]:
# Multi-Agent Orchestration
from typing import TypedDict, Optional, Literal
from langgraph.graph import StateGraph, END

In [35]:
class AgentState(TypedDict):
    input_type: Literal["url", "pdf", "text"]
    value: str
    text: Optional[str]
    error: Optional[str]
    next: Optional[str]
    label: Optional[str]
    confidence: Optional[float]
    explanation: Optional[str]
    fallback_used: Optional[bool]

In [36]:
# InputHandlerAgent
def InputHandlerAgent(state: AgentState):
    text = extract_text(state["input_type"], state["value"])
    return {**state, "text": text}

In [37]:
# PlannerAgent
def PlannerAgent(state: AgentState):
    if len(state.get("text","").strip()) < 50:
        return {**state, "next": "FallbackSearchAgent"}
    return {**state, "next": "ToolRouterAgent"}

In [38]:
# FallbackSearchAgent
def FallbackSearchAgent(state: AgentState):
    query = state.get("value")
    try:
        result = next(DDGS().text(query))
        summary = result["body"]
    except Exception:
        summary = "No fallback results found."
    return {**state, "text": summary, "fallback_used": True}

In [39]:
# ToolRouterAgent
def ToolRouterAgent(state: AgentState):
    return {**state, "next": "ExecutorAgent"}

In [40]:
# ClassifierAgent
def ClassifierAgent(state: AgentState):
    inputs = roberta_tokenizer(state['text'], return_tensors="pt", truncation=True, padding='max_length', max_length=256).to(device)
    with torch.no_grad():
        logits = roberta_model(**inputs).logits
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()
    label = "Real" if pred == 0 else "Fake"
    confidence = probs[0][pred].item()
    return {**state, "label": label, "confidence": confidence}

In [41]:
# ExplanationAgent
def ExplanationAgent(state: AgentState):
    prompt = f"Explain why this news might be {state['label'].lower()} in one sentence: {state['text']}"
    inputs = flan_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    output_ids = flan_model.generate(inputs['input_ids'], max_new_tokens=100)
    explanation = flan_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return {**state, "explanation": explanation}

In [42]:
# ExecutorAgent
def ExecutorAgent(state: AgentState):
    state = ClassifierAgent(state)
    state = ExplanationAgent(state)
    return state

In [43]:
# Build LangGraph
graph = StateGraph(AgentState)
graph.add_node("InputHandlerAgent", InputHandlerAgent)
graph.add_node("PlannerAgent", PlannerAgent)
graph.add_node("FallbackSearchAgent", FallbackSearchAgent)
graph.add_node("ToolRouterAgent", ToolRouterAgent)
graph.add_node("ExecutorAgent", ExecutorAgent)

In [44]:
graph.set_entry_point("InputHandlerAgent")
graph.add_edge("InputHandlerAgent", "PlannerAgent")
graph.add_conditional_edges("PlannerAgent", lambda s: s["next"], {
    "FallbackSearchAgent": "FallbackSearchAgent",
    "ToolRouterAgent": "ToolRouterAgent"
})
graph.add_edge("FallbackSearchAgent", "ToolRouterAgent")
graph.add_edge("ToolRouterAgent", "ExecutorAgent")
graph.add_edge("ExecutorAgent", END)

In [45]:
runnable = graph.compile()

In [46]:
# Example Usage
result = runnable.invoke({
    "input_type": "text",
    "value": "The moon landing was staged in a Hollywood studio."
})

print("\nPrediction Results")
print(f"Label: {result['label']}")
print(f"Confidence: {result['confidence']:.2f}")
print(f"Explanation: {result['explanation']}")
if result.get("fallback_used"):
    print("Used fallback search")


Prediction Results
Label: Fake
Confidence: 0.87
Explanation: The moon landing was staged in a Hollywood studio.
